# Model 2- LSTM

**Long Short-Term Memory (LSTM)** is an advanced type of recurrent neural network (RNN) designed to overcome the challenges of learning long-term dependencies in sequential data. It features memory cells with specialized gates—forget, input, and output—that enable it to selectively retain and update information over extended sequences. This capability makes LSTMs ideal for tasks requiring nuanced understanding of context over time, such as time-series prediction, natural language processing, and speech recognition. Key advantages include their ability to mitigate the vanishing gradient problem, ensuring stable gradient flow during training, and their flexibility in handling variable-length sequences, which contributes to their widespread use in diverse fields of deep learning and artificial intelligence.


## 1.Data Preprocessing
Data preprocessing transforms raw data into a clean and usable format by handling missing values, outliers, and ensuring consistent data scales through normalization or standardization. It also includes feature extraction and selection to enhance dataset quality. This step is essential for efficient and accurate data analysis or machine learning model performance.

In [3]:
# importing all the necessary libraries
import pandas as pd
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from scipy.sparse import hstack
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


In [4]:
# Reading and displaying the first 10 rows of the CSV file
df=pd.read_csv('fake_job_postings.csv')
df.head(10)

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0
5,6,Accounting Clerk,"US, MD,",NaN,NaN,NaN,Job OverviewApex is an environmental consultin...,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,0
6,7,Head of Content (m/f),"DE, BE, Berlin",ANDROIDPIT,20000-28000,"Founded in 2009, the Fonpit AG rose with its i...",Your Responsibilities: Manage the English-spea...,Your Know-How: ...,Your Benefits: Being part of a fast-growing co...,0,1,1,Full-time,Mid-Senior level,Master's Degree,Online Media,Management,0
7,8,Lead Guest Service Specialist,"US, CA, San Francisco",NaN,NaN,Airenvy’s mission is to provide lucrative yet ...,Who is Airenvy?Hey there! We are seasoned entr...,"Experience with CRM software, live chat, and p...",Competitive Pay. You'll be able to eat steak e...,0,1,1,NaN,NaN,NaN,NaN,NaN,0
8,9,HP BSM SME,"US, FL, Pensacola",NaN,NaN,Solutions3 is a woman-owned small business who...,Implementation/Configuration/Testing/Training ...,MUST BE A US CITIZEN.An active TS/SCI clearanc...,NaN,0,1,1,Full-time,Associate,NaN,Information Technology and Services,NaN,0
9,10,Customer Service Associate - Part Time,"US, AZ, Phoenix",NaN,NaN,"Novitex Enterprise Solutions, formerly Pitney ...",The Customer Service Associate will be based i...,Minimum Requirements:Minimum of 6 months custo...,NaN,0,1,0,Part-time,Entry level,High School or equivalent,Financial Services,Customer Service,0


In [5]:
# Fill missing values in text columns with empty strings
text_columns = ['description', 'requirements', 'company_profile', 'title', 'location', 'department', 'salary_range', 'employment_type', 'required_experience', 'required_education', 'industry', 'function']
df[text_columns] = df[text_columns].fillna('')

In [6]:
# Combine text columns into a single column for vectorization
df['combined_text'] = df[text_columns].apply(lambda x: ' '.join(x), axis=1)

In [7]:
# Simplified text preprocessing
df['combined_text'] = df['combined_text'].str.lower()
df['combined_text'] = df['combined_text'].apply(lambda x: re.sub(r'http\S+|www\S+|https\S+', '', x))
df['combined_text'] = df['combined_text'].str.replace('\n', ' ')
df['combined_text'] = df['combined_text'].apply(lambda x: re.sub(r'\w*\d\w*', '', x))
df['combined_text'] = df['combined_text'].str.strip()
df['combined_text'] = df['combined_text'].apply(lambda x: re.sub(' +', ' ', x))
df['combined_text'] = df['combined_text'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))

In [8]:
# Tokenize the text data
tokenizer = Tokenizer(num_words=5000)  # Limit to the top 5000 words
tokenizer.fit_on_texts(df['combined_text'])
X_text = tokenizer.texts_to_sequences(df['combined_text'])

# Pad sequences to the same length
max_length = 100
X_text_padded = pad_sequences(X_text, maxlen=max_length, padding='post')

In [9]:
# List of non-text columns to be used as features
non_text_columns = ['telecommuting', 'has_company_logo', 'has_questions', 'location', 'department', 'salary_range', 'employment_type', 'required_experience', 'required_education', 'industry', 'function']

In [10]:
# Fill missing values in non-text columns with a placeholder
df[non_text_columns] = df[non_text_columns].fillna('missing')

In [11]:
# OneHotEncode the categorical non-text columns
categorical_columns = ['location', 'department', 'salary_range', 'employment_type', 'required_experience', 'required_education', 'industry', 'function']
encoder = OneHotEncoder()
X_non_text_encoded = encoder.fit_transform(df[categorical_columns])

In [12]:
# Scale non-categorical non-text columns
scaler = StandardScaler()
X_non_text_scaled = scaler.fit_transform(df[['telecommuting', 'has_company_logo', 'has_questions']])

In [13]:
# Combine non-categorical and categorical features
from scipy.sparse import hstack
X_non_text = hstack([X_non_text_encoded, X_non_text_scaled]).toarray()

In [14]:
# Combine the text and non-text features
X_combined = np.hstack((X_text_padded, X_non_text))

## Model Building
Using LSTM involves preprocessing sequential data, stacking LSTM layers to capture temporal patterns, and training with backpropagation. Evaluation metrics like accuracy measure performance, with hyperparameter tuning optimizing model effectiveness for tasks like time series or NLP.

The approach used in the provided is tailored for training an LSTM model on sequential data, specifically for regression tasks. Here's a concise summary of why each step was implemented:

1. **Subset of Data for Quick Testing**:
   - **Reason**: Allows rapid initial testing and debugging of the LSTM model code on a smaller portion of the dataset (`X_subset`, `y_subset`).

2. **Splitting Data into Training, Testing, and Validation Sets**:
   - **Reason**: Ensures robust evaluation of the LSTM model’s performance by separating data into distinct training (`X_train`, `y_train`), validation (`X_val`, `y_val`), and test (`X_test`, `y_test`) sets.

3. **Defining the LSTM Model**:
   - **Reason**: LSTM models are well-suited for capturing sequential dependencies in data, making them appropriate for tasks involving time-series or sequential patterns.

4. **Reshaping Input Data to 3D**:
   - **Reason**: LSTM models in TensorFlow require input data in a three-dimensional format (`[samples, time steps, features]`), ensuring compatibility with the model’s architecture.

5. **Training the Model**:
   - **Reason**: Optimizes the LSTM model’s parameters on the training data (`X_train`, `y_train`) and evaluates its performance on the validation set (`X_val`, `y_val`) to monitor and improve generalization ability.

In essence, this approach systematically prepares, defines, and trains an LSTM model using TensorFlow, leveraging its capabilities to handle sequential data while ensuring effective evaluation and optimization through proper data splitting and reshaping.

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Use a smaller subset of the data for quick testing
X_subset, _, y_subset, _ = train_test_split(
    X_combined, df['fraudulent'], test_size=0.8, random_state=42
)

# Split the data into training, testing, and validation sets
X_train, X_temp, y_train, y_temp = train_test_split(
    X_subset, y_subset, test_size=0.3, random_state=42
)
X_test, X_val, y_test, y_val = train_test_split(
    X_temp, y_temp, test_size=0.33, random_state=42
)  # 0.33 * 0.3 = 0.10

# Define the model
model = Sequential()
model.add(LSTM(10, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(10))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Reshape the input data to 3D
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))

# Train the model
model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=16,
    verbose=1,
)


C:\DELL\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 1264s 8s/step - loss: 0.0488 - val_loss: 0.0369
Epoch 2/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 1003s 6s/step - loss: 0.0455 - val_loss: 0.0355
Epoch 3/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 872s 6s/step - loss: 0.0495 - val_loss: 0.0352
Epoch 4/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 618s 4s/step - loss: 0.0437 - val_loss: 0.0365
Epoch 5/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 633s 4s/step - loss: 0.0511 - val_loss: 0.0346
Epoch 6/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 642s 4s/step - loss: 0.0451 - val_loss: 0.0350
Epoch 7/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 639s 4s/step - loss: 0.0440 - val_loss: 0.0365
Epoch 8/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 637s 4s/step - loss: 0.0432 - val_loss: 0.0383
Epoch 9/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 638s 4s/step - loss: 0.0473 - val_loss: 0.0361
Epoch 10/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 641s 4s/step - loss: 0.0527 - val_loss: 0.0354


## Model Evaluation
Model evaluation involves assessing a trained model's performance using various metrics to determine its effectiveness and generalization ability. Common metrics for classification tasks include accuracy, precision, recall, F1 score, and AUC-ROC, while regression tasks often use mean squared error (MSE), mean absolute error (MAE), and R-squared. The evaluation process typically includes splitting the data into training and testing sets, training the model on the training set, and evaluating it on the test set to ensure the model performs well on unseen data. Cross-validation can also be used for more robust evaluation.

In [19]:
from sklearn.metrics import mean_absolute_error

# Evaluate the model on the test set
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss (MSE): {test_loss}')

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE): {mae}')

23/23 ━━━━━━━━━━━━━━━━━━━━ 10s 417ms/step - loss: 0.0476
Test Loss (MSE): 0.04049093276262283
23/23 ━━━━━━━━━━━━━━━━━━━━ 46s 444ms/step
Mean Absolute Error (MAE): 0.07598628434495258


## Conclusion
The model shows a low Mean Squared Error (MSE) of 0.0411 and a Mean Absolute Error (MAE) of 0.0799 on the test data, indicating good overall performance with minimal error in predictions. These results suggest that the model has a strong ability to predict values close to the actual targets, making it effective for the given regression task.

### LSTM Model
- **Test Loss (MSE)**: 0.0411
- **Mean Absolute Error (MAE)**: 0.0799


### Comparison:

2. **LSTM Model**:
   - **Plus**: Low Mean Squared Error (MSE) and Mean Absolute Error (MAE), indicating good predictive performance.
   - **minus**: LSTM models are typically more complex and resource-intensive to train and deploy.
   - **suitable scenario**: Time series data or sequential data where capturing the temporal dependencies is crucial.

### Recommendation:

If achieving high accuracy across all classes is paramount, particularly with a noticeable class imbalance, **Logistic Regression** remains a strong contender, despite its tendency to underperform on minority classes.

In scenarios involving sequential or time-series data, where capturing intricate dependencies is crucial, opting for an **LSTM Model** would be advantageous.

When dealing with textual data or situations requiring nuanced understanding of context, leveraging a **BERT-Based Model** proves highly effective.

When analyzing metrics, the Logistic Regression model consistently demonstrates superior performance across accuracy and weighted averages. However, depending on the complexity and specificity of the task at hand, either the LSTM or BERT model might offer more suitable solutions.
